In [1]:
import numpy as np

tank_param_bounds: dict = {
    # tank-0
    "t0_is": {"min": 0.01, "max": 100},
    "t0_boc": {"min": 0.1, "max": 0.5},
    "t0_soc_uo": {"min": 0.1, "max": 0.5},
    "t0_soc_lo": {"min": 0.1, "max": 0.5},
    "t0_soh_uo": {"min": 75, "max": 100},
    "t0_soh_lo": {"min": 0, "max": 50},

    # tank-1
    "t1_is": {"min": 0.01, "max": 100},
    "t1_boc": {"min": 0.01, "max": 0.5},
    "t1_soc": {"min": 0.01, "max": 0.5},
    "t1_soh": {"min": 0, "max": 100},

    # tank-2
    "t2_is": {"min": 0.01, "max": 100},
    "t2_boc": {"min": 0.01, "max": 0.5},
    "t2_soc": {"min": 0.01, "max": 0.5},
    "t2_soh": {"min": 0, "max": 100},

    # tank-3
    "t3_is": {"min": 0.01, "max": 100},
    "t3_soc": {"min": 0.01, "max": 0.5}
}
tank_lb: np.ndarray = np.array([
    # [Tank-0] 
    tank_param_bounds['t0_is']['min'],
    tank_param_bounds['t0_boc']['min'],
    tank_param_bounds['t0_soc_uo']['min'],
    tank_param_bounds['t0_soc_lo']['min'],
    tank_param_bounds['t0_soh_uo']['min'],
    tank_param_bounds['t0_soh_lo']['min'],

    # [Tank-1]
    tank_param_bounds['t1_is']['min'],
    tank_param_bounds['t1_boc']['min'],
    tank_param_bounds['t1_soc']['min'],
    tank_param_bounds['t1_soh']['min'],

    # [Tank-2]
    tank_param_bounds['t2_is']['min'],
    tank_param_bounds['t2_boc']['min'],
    tank_param_bounds['t2_soc']['min'],
    tank_param_bounds['t2_soh']['min'],

    # [Tank-3]
    tank_param_bounds['t3_is']['min'],
    tank_param_bounds['t3_soc']['min'],

])
muskingum_param_bound: dict = {
    "k": {"min": 0, "max": 5},
    "x": {"min": 0, "max": 0.5}

}
muskingum_lb: np.ndarray = np.array([
    muskingum_param_bound["k"]["min"],
    muskingum_param_bound["x"]["min"],
])
# muskingum parameter upper bounds
muskingum_ub: np.ndarray = np.array([
    muskingum_param_bound["k"]["max"],
    muskingum_param_bound["x"]["max"],
])

basin_default = tank_lb
channel_default = 0.5 * (muskingum_lb + muskingum_ub)
parsed_node = dict()
print(basin_default)

[1.0e-02 1.0e-01 1.0e-01 1.0e-01 7.5e+01 0.0e+00 1.0e-02 1.0e-02 1.0e-02
 0.0e+00 1.0e-02 1.0e-02 1.0e-02 0.0e+00 1.0e-02 1.0e-02]


In [2]:
basin = open('CedarCreek.basin', 'r').read()
print(basin)

Basin: CedarCreek
     Description: HMS Model For Cedar Creek
     Last Modified Date: 1 November 2010
     Last Modified Time: 18:18:23
     Version: 4.10
     Filepath Separator: \
     Unit System: English
     Grid Cell File: CedarCreek.mod
     Missing Flow To Zero: No
     Enable Flow Ratio: No
     Compute Local Flow At Junctions: No
     Unregulated Output Required: No

     Enable Sediment Routing: No

     Enable Quality Routing: No
End:

Subbasin: W310
     Last Modified Date: 19 February 2023
     Last Modified Time: 02:40:17
     Canvas X: 658813.695603162
     Canvas Y: 4566675.75032761
     Label X: 16.0
     Label Y: 16.0
     Area: 6.3298
     Downstream: Outlet1

     Discretization: None
     File: CedarCreek.sqlite

     Canopy: None
     Allow Simultaneous Precip Et: No
     Plant Uptake Method: None

     Surface: None

     LossRate: SCS
     Percent Impervious Area: 0.0
     Curve Number: 78.870
     Initial Abstraction: 0

     Transform: SCS
     Lag: 143.19
 

In [3]:
def extract_data(file_content):
    sections = file_content.split("End:\n")
    data = []

    for section in sections:
        lines = section.strip().split("\n")
        section_data = {}
        current_key = None

        for line in lines:
            if ":" in line:
                key, value = line.split(":", 1)
                section_data[key.strip()] = value.strip()
                current_key = key.strip()
            elif line.strip() and current_key:
                section_data[current_key] += " " + line.strip()

        if section_data:
            data.append(section_data)

    return data

In [4]:
data = extract_data(basin)
for section in data:
    print(section)

{'Basin': 'CedarCreek', 'Description': 'HMS Model For Cedar Creek', 'Last Modified Date': '1 November 2010', 'Last Modified Time': '18:18:23', 'Version': '4.10', 'Filepath Separator': '\\', 'Unit System': 'English', 'Grid Cell File': 'CedarCreek.mod', 'Missing Flow To Zero': 'No', 'Enable Flow Ratio': 'No', 'Compute Local Flow At Junctions': 'No', 'Unregulated Output Required': 'No', 'Enable Sediment Routing': 'No', 'Enable Quality Routing': 'No'}
{'Subbasin': 'W310', 'Last Modified Date': '19 February 2023', 'Last Modified Time': '02:40:17', 'Canvas X': '658813.695603162', 'Canvas Y': '4566675.75032761', 'Label X': '16.0', 'Label Y': '16.0', 'Area': '6.3298', 'Downstream': 'Outlet1', 'Discretization': 'None', 'File': 'CedarCreek.sqlite', 'Canopy': 'None', 'Allow Simultaneous Precip Et': 'No', 'Plant Uptake Method': 'None', 'Surface': 'None', 'LossRate': 'SCS', 'Percent Impervious Area': '0.0', 'Curve Number': '78.870', 'Initial Abstraction': '0', 'Transform': 'SCS', 'Lag': '143.19', '

In [5]:
basin = {
    'basin_def': {

    },
    'root_node': [

    ]
}

In [14]:
TANK_PARAMETER_ORDER: list = [
    # T-0
    't0_is', 't0_boc', 't0_soc_uo', 't0_soc_lo', 't0_soh_uo', 't0_soh_lo',
    # T-1
    't1_is', 't1_boc', 't1_soc', 't1_soh',
    # T-2
    't2_is', 't2_boc', 't2_soc', 't2_soh',
    # T-3
    't3_is', 't3_soc'
]


def muskingum_param_list2dict(parameters: list) -> dict:
    return {
        "k": parameters[0],
        "x": parameters[1],
    }


def tank_param_list2dict(parameters: list) -> dict:
    # converts flattened parameter to a dict based on TANK_PARAMETER_ORDER

    return {
        parameter_name: parameters[i]
        for i, parameter_name in enumerate(TANK_PARAMETER_ORDER)
    }

In [21]:
sel_nodes = ["Subbasin", "Reach", "Junction", "Sink"]
generel_props = ["Downstream", "Area", "Computation Point"]
numeric_props = ["Area"]

basin_def = {}
for section in data:
    for k,v in section.items():
        component = {}
        if k == 'Subbasin':
            component['parameters'] = tank_param_list2dict(list(basin_default))
        if k == 'Reach':
            component['parameters'] = muskingum_param_list2dict(list(channel_default))
        
        
        
    
    # for s in section:
    #     if s in sel_nodes:
    #         component = {'type': s}
    # 
    #         if s == 'Subbasin':
    #             component['parameters'] = tank_param_list2dict(list(basin_default))
    # 
    #         if s == 'Reach':
    #             component['parameters'] = muskingum_param_list2dict(list(channel_default))
    #             
    #         basin_def[section[s]] = component
        
        

basin['basin_def'] = basin_def

for node in basin['basin_def']:
    ds = basin['basin_def'][node].get('downstream', None)

    if ds is None:
        #  this is root node || needs to be changed
        # no basin will have multiple root node
        if basin.get('root_node', None) is None:
            basin['root_node'] = [node]
        else:
            basin['root_node'].append(node)

    else:
        if basin['basin_def'][ds].get('upstream', None) == None:

            basin['basin_def'][ds]['upstream'] = [node]
        else:
            basin['basin_def'][ds]['upstream'].append(node)

print(basin)

Basin CedarCreek
Description HMS Model For Cedar Creek
Last Modified Date 1 November 2010
Last Modified Time 18:18:23
Version 4.10
Filepath Separator \
Unit System English
Grid Cell File CedarCreek.mod
Missing Flow To Zero No
Enable Flow Ratio No
Compute Local Flow At Junctions No
Unregulated Output Required No
Enable Sediment Routing No
Enable Quality Routing No
Subbasin W310
Last Modified Date 19 February 2023
Last Modified Time 02:40:17
Canvas X 658813.695603162
Canvas Y 4566675.75032761
Label X 16.0
Label Y 16.0
Area 6.3298
Downstream Outlet1
Discretization None
File CedarCreek.sqlite
Canopy None
Allow Simultaneous Precip Et No
Plant Uptake Method None
Surface None
LossRate SCS
Percent Impervious Area 0.0
Curve Number 78.870
Initial Abstraction 0
Transform SCS
Lag 143.19
Unitgraph Type STANDARD
Baseflow None
Subbasin W300
Last Modified Date 19 February 2023
Last Modified Time 02:40:17
Canvas X 648943.207364027
Canvas Y 4568908.24597557
Label X 16.0
Label Y 16.0
Area 27.969
Downstre